<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Airline Tweets Sentiment Analysis Lab

_Authors: Phillippa Thomson (NYC)_

---

You are going to be analyzing tweets about airlines.  These have been hand-tagged with sentiment.  There are three categories: positive, neutral, and negative.

Use VADER to calculate sentiment for each tweet, and see if you can correctly predict the hand-tagged sentiment.

What is the accuracy?  Print out a heatmap to see where your model performs well, and where it performs poorly.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, \
precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tweets = pd.read_csv("../../data/Tweets.csv", encoding='latin')

In [ ]:
tweets.head()

### 1. Preview the airline_sentiment column.
- What percentage of reviews are positive, neutral, and negative?

In [ ]:
tweets['airline_sentiment'].value_counts() / len(tweets)

### 2. Load in the Sentiment IntensityAnalyzer from Vader and add compound, negative, neutral, and positive scores into the DataFrame.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

compound= []
neg = []
neu = []
pos = []
for tweet in tweets['text']:
    sent = sia.polarity_scores(tweet)
    compound.append(sent['compound'])
    neg.append(sent['neg'])
    neu.append(sent['neu'])
    pos.append(sent['pos'])

In [ ]:
tweets['compound'] = compound
tweets['neg'] = neg
tweets['neu'] = neu
tweets['pos'] = pos

In [ ]:
tweets.head()

### 3. Store airline_sentiment in y to use as labels and create an appropriate feature matrix, X.

In [ ]:
y = tweets['airline_sentiment']
X = tweets[['compound', 'neg', 'neu', 'pos']]

### 4. Fit a model of your choice to predict airline_sentient and cross-validate.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
rf.fit(X,y)

In [ ]:
cross_val_score(rf, X, y)
# versus the baseline (63%), this is a little weak.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.30,
                                                        random_state=14)
rf.fit(X_train,y_train)

### 5. Display the confusion matrix.
- What reviews are difficult to identify?

In [ ]:

conmat = np.array(confusion_matrix(y_test, rf.predict(X_test)))
confusion = pd.DataFrame(conmat, index=['negative', 'neutral', 'positive'],\
                     columns=['Pred neg', 'Pred neutral', 'Pred pos'])

plt.figure(figsize = (6,6))
heat = sns.heatmap(confusion, annot=True, annot_kws={"size": 20},cmap='Blues',fmt='g', cbar=False)
plt.xticks(rotation=0, fontsize=14)
plt.yticks(fontsize=14)
plt.title("Confusion Matrix", fontsize=20)


### 6. Print the classification report and discuss the characteristics of the model.

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

The model does ok with negative tweets (the predominant class) but quite poorly with neutral.

To put this in perspective, human concordance, the probability that two people assign the same sentiment to an observation is usually around 70%-80% our baseline is at 63%. Even small increases in accuracy quickly move us towards a theoretical maximum in performance.